# Citrics Documentation - Data Collection

The data used in the project was from a number of different sources. Merging data from these data sources required making a uniform naming convention for each city, the data science team decided to use a format of `city_name ST`, where 'ST' is the postal abbreviation used for each state. The combined data set exceeds the file size that we can add to github, so that csv file isn't availabe, but could be pulled from our MongoDB.

The most comprehensive data set that we built off of was the US Census Data (https://www.census.gov/acs/www/data/data-tables-and-tools/data-profiles/2018/).

## Census Data
#### Census Data Collection Methodology

The process for collecting the Census data was fairly manual. We went to the Census Website:  https://data.census.gov/cedsci/table?q=&d=ACS%205-Year%20Estimates%20Data%20Profiles&table=DP02&tid=ACSDP5Y2017.DP02&lastDisplayedRow=39&hidePreview=true&g=0100000US.160000 and performed the following steps

1) Select the Geographies option from the navigation bar

2) Clear anything in the "Selected Geographies:" across the bottom of the page

3) From the remaining Geography column, scroll and select "Place", a "Place" column will appear

4) Scroll and select which state you need the data for, that state's name will appear in a new column
- Note: for some of the Tables you will be able to select multiple states at a time to speed up this process

5) From that column, select the first option "all places in state_name"

<img src = 'https://raw.githubusercontent.com/Lambda-School-Labs/city-data-comparison-ds/master/images/census-select-geographies.PNG'>
6) Hit the close button in the bottom right corner

7) Select "Download Table"

8) Select that you want the file as a csv with the date you want

- NOTE: The population of each city from 2010 through the most recently available year, came from the above process as well, but at step 8 select an earlier year.

9) You'll then have a zip file with the data for that state/time

10) Repeat 1-9 until you have all states for that particular table

11) Then from the top-left corner of the page, select the "Tables" link

<img src = 'https://raw.githubusercontent.com/Lambda-School-Labs/city-data-comparison-ds/master/images/census-tables-highlighted.PNG'>
12) Select another table (in total, you'll need DP02, DP03, DP04, and DP05) 

13) Select "Customize Table" from the top-right corner of the page and repeat steps 1-12 until you've collected the entirety of the data needed

#### Feature Selection

We also went through a manual process of looking at the 2000+ columns in this combined data set. At the DS Team's discretion, we selected the following features:

In [5]:
import pandas as pd

In [6]:
cenesus_column_headers = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/city-data-comparison-ds/master/data-help/census-columns.csv')

In [10]:
print(cenesus_column_headers.shape)
cenesus_column_headers.head()

(381, 1)


,column_headers
0,DP02_0001E
1,DP02_0002E
2,DP02_0002PE
3,DP02_0004PE
4,DP02_0017E


#### Merging

The `GEO_ID` field was used as the key to merge assorted Census DataFrames on. 

## Walk Score Data Collection Methodology

Walk Score is a service that offers a uniform scoring methodology to rate cities overall as well as specific addresses on their walkability, how bike-friendly they are, and a score for the public transit available. Using a third party for this because we do not have to make our own methodology for these types of figures.

About Walk Score:
https://www.walkscore.com/professional/research.php

Source of the data tables:
https://www.walkscore.com/cities-and-neighborhoods/states/

Trademark Guidelines:
https://www.walkscore.com/trademark-use.shtml

In [1]:
import pandas as pd

In [33]:
us_state_abbrev = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS',
                   'KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC',
                   'ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [37]:
# Empty DataFrame to add to in the loop below
df = pd.DataFrame()

for i in us_state_abbrev:
    df_i = pd.read_html('https://www.walkscore.com/' + i)[0]
    df_i['State'] = i
    df = pd.concat([df, df_i])

This is a simple loop that uses the `pd.read_html` function and a list of state abbreviations to scrape the summary scores for all major cities in their publicly-available data tables. 

In [39]:
df.head()

,City,Zip Code,Walk Score,Transit Score,Bike Score,Population,State
0,Birmingham (the largest city in Alabama),35211.0,35,25,31,212237,AL
1,Montgomery,36109.0,27,16,38,205764,AL
2,Mobile,36605.0,33,--,39,195111,AL
3,Huntsville,35810.0,23,13,40,180105,AL
4,Tuscaloosa,NaN,33,--,37,90468,AL


### Real Estate Data Collection Methodology - Zillow

The process for collecting Zillow's real estate data was similar to that of the Census data, go to their website: https://www.zillow.com/research/data/ and perform the following steps:

**Home Values**

1) Housing data is collected from the the top option on this page, we collected five "Data Types" from the drop down (`ZHVI 1-Bedroom Time Series ($)`, `ZHVI 2-Bedroom Time Series ($)`, through 5-Bedroom+)

2) And for Geography select "City"

3) Download, this gives you a csv file

4) Repeat steps 1-3 for each Data Type

### Latitude - Longitude Data Source

For the majority of lat-long locations, we got a csv file from: https://simplemaps.com/data/us-cities

For those cities not listed we used the Mapbox API: https://docs.mapbox.com/api/

In [ ]:
def get_data(url):
    """
    Where url is a city name.
    
    Going through nested arrays:
        - state ID
        - city name - entire "Place" value from the census data
        - GEOID
    """
    
    get = requests.get(f"https://api.mapbox.com/geocoding/v5/mapbox.places/{url[1]}.json?types=place&country=US&access_token=GET-YOUR-OWN-DANG-TOKEN").json()
    try:
        check_first = get['features'][0]['context'][0]['short_code'].split('-')[1] #checking that the city is in the right state
        if check_first == url[0]:
            locs = get['features'][0]['center']
            lat = locs[1]
            lng = locs[0]
            place = get['features'][0]['place_name']
        else:
            lat = True
            lng = True
            place = True
    except:
        lat = False
        lng = False
        place = False
    res = [url[0], url[1], url[2], lat, lng, place]
    return(res)

### Weather Data Collection Methodology

Source for the weather data: https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted

Using the latitude/longitude data collected above, we used a function in NOAA's api called 'extent' which effectively draws a bounding box around a location and returns all weather stations in that area, sort by data coverage and you have a weather station ID to use to pull other data from.

In [ ]:
def stations(i):
    lat = dataset['lat'].iloc[i]
    lng = dataset['lng'].iloc[i]
    
    # Using one half a degree in each direction from each city (~35 miles)
    def box_size(lng):
        if lng < -170: # Island chain off of Alaska
            return 7
        elif lng > -125: #Basically all of the lower 48
            return 1
        else:
            return 2.5 # The rest of Alaska
        
    coord_len = box_size(lng)
        
    
    N_coord = lat + coord_len
    S_coord = lat - coord_len
    W_coord = lng - coord_len
    E_coord = lng + coord_len
    
    # Note that for this particular API the coordinates MUST be in this order: SWNE
    boundingbox = 'extent=' + str(S_coord) + ',' + str(W_coord) + ',' + str(N_coord) + ',' + str(E_coord)
    
    
    # Search stations within the 'boundingbox' defined above,
    # only for stations that record TEMP and SUN,
    # data must exist later than December 2018 for 'Global Summary of the Month', 
    # limited to 500 stations (shouldn't be a constraint, basically just saying show all)
    get_stations = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?' + boundingbox + '&datacategoryid=TEMP&startdate=2019-12-01&datasetid=GSOM&limit=500',
                       headers={'token': 'GET-YOUR-OWN-DANG-TOKEN'})
    station_json = get_stations.json()
    try:
        station_data = station_json['results']
        df_station_data = pd.DataFrame(station_data)
        df_station_data = df_station_data.sort_values(by='datacoverage', ascending=0)
        search_station_0 = df_station_data['id'].iloc[0]
        return search_station_0
    except:
        pass


all_stations = []

t1 = time.perf_counter()
for i in range(1, 300): #I had to break the pull up into smaller chunks and run multiple times to get it to work.
    all_stations.append(stations(i))
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Drop the duplicates from the list of stations and return the station info for the exact lat/lng for which the data you'll pull is associated:

In [ ]:
df_locations = pd.DataFrame()

t1 = time.perf_counter()

for i in [0]:

    station_n = df_as2['station'][i]

    if station_n == None:
        pass
    else:
        get = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations/' + station_n,
                           headers={'token':'GET-YOUR-OWN-DANG-TOKEN'})

        locations_json = get.json()
        df = pd.DataFrame.from_dict(locations_json, orient='index').T
        
        df_locations = pd.concat([df_locations, df], axis=0)

    
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Now take the list of station IDs and get the actual weather data:

In [ ]:
df_locations = pd.DataFrame()

t1 = time.perf_counter()

for i in range(550, 684):

    station_n = df_locations_all['id'][i]

    if station_n == None:
        pass
    else:
        get = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GSOM&datatypeid=TAVG,TMAX,TMIN,PRCP,SNOW,PSUN&stationid=' + station_n + '&startdate=2016-01-01&enddate=2019-12-01&units=standard&limit=500',
                           headers={'token':'GET-YOUR-OWN-DANG-TOKEN'})

        weather_json = get.json()
        weather_data = weather_json['results']
        weather_df = pd.DataFrame(weather_data)

        df_pivot = weather_df.pivot(index='date', columns='datatype', values='value')

        headers=df_pivot.columns

        df_pivot['date'] = pd.to_datetime(df_pivot.index, infer_datetime_format=True)
        df_pivot['Month'] = df_pivot['date'].dt.month.astype(str)

        df_columns=['station', 'Month', 'PRCP-AVG', 'SNOW-AVG', 'TAVG-AVG', 'TMIN-AVG', 'TMAX-AVG']

        if('PRCP' in headers):
            df_pivot['PRCP-AVG'] = df_pivot.groupby(['Month'])['PRCP'].transform('mean')
        else:
            df_pivot['PRCP-AVG'] = np.nan

        if('SNOW' in headers):
            df_pivot['SNOW-AVG'] = df_pivot.groupby(['Month'])['SNOW'].transform('mean')
        else:
            df_pivot['SNOW-AVG'] = np.nan

        if('TAVG' in headers):
            df_pivot['TAVG-AVG'] = df_pivot.groupby(['Month'])['TAVG'].transform('mean')
        else:
            df_pivot['TAVG-AVG'] = np.nan

        if('TMAX' in headers):
            df_pivot['TMAX-AVG'] = df_pivot.groupby(['Month'])['TMAX'].transform('mean')
        else:
            df_pivot['TMAX-AVG'] = np.nan

        if('TMIN' in headers):
            df_pivot['TMIN-AVG'] = df_pivot.groupby(['Month'])['TMIN'].transform('mean')
        else:
            df_pivot['TMIN-AVG'] = np.nan

        df_pivot['station'] = station_n
        df_output = df_pivot[df_columns].copy()
        df_output = df_output.reset_index()
        df_output = df_output.drop('date', axis=1)
        df_output = df_output.drop_duplicates()
        df_output['Month'] = df_output['Month'].astype(int)
        df_output = df_output.sort_values(by='Month', ascending=1)
        df_output = df_output.set_index('Month')

        x = df_output.to_dict()  
        df = pd.DataFrame()

        df_output
        if len(df_output.index) == 12:
            df['station'] = [x['station'][1]]
        else:
            headers = []

        if('PRCP' in headers):
            df['PRCP-AVG-1'] = [x['PRCP-AVG'][1]]
            df['PRCP-AVG-2'] = [x['PRCP-AVG'][2]]
            df['PRCP-AVG-3'] = [x['PRCP-AVG'][3]]
            df['PRCP-AVG-4'] = [x['PRCP-AVG'][4]]
            df['PRCP-AVG-5'] = [x['PRCP-AVG'][5]]
            df['PRCP-AVG-6'] = [x['PRCP-AVG'][6]]
            df['PRCP-AVG-7'] = [x['PRCP-AVG'][7]]
            df['PRCP-AVG-8'] = [x['PRCP-AVG'][8]]
            df['PRCP-AVG-9'] = [x['PRCP-AVG'][9]]
            df['PRCP-AVG-10'] = [x['PRCP-AVG'][10]]
            df['PRCP-AVG-11'] = [x['PRCP-AVG'][11]]
            df['PRCP-AVG-12'] = [x['PRCP-AVG'][12]]
        else:
            df['PRCP-AVG-1'] = np.nan
            df['PRCP-AVG-2'] = np.nan
            df['PRCP-AVG-3'] = np.nan
            df['PRCP-AVG-4'] = np.nan
            df['PRCP-AVG-5'] = np.nan
            df['PRCP-AVG-6'] = np.nan
            df['PRCP-AVG-7'] = np.nan
            df['PRCP-AVG-8'] = np.nan
            df['PRCP-AVG-9'] = np.nan
            df['PRCP-AVG-10'] = np.nan
            df['PRCP-AVG-11'] = np.nan
            df['PRCP-AVG-12'] = np.nan


        if('SNOW' in headers):
            df['SNOW-AVG-1'] = [x['SNOW-AVG'][1]]
            df['SNOW-AVG-2'] = [x['SNOW-AVG'][2]]
            df['SNOW-AVG-3'] = [x['SNOW-AVG'][3]]
            df['SNOW-AVG-4'] = [x['SNOW-AVG'][4]]
            df['SNOW-AVG-5'] = [x['SNOW-AVG'][5]]
            df['SNOW-AVG-6'] = [x['SNOW-AVG'][6]]
            df['SNOW-AVG-7'] = [x['SNOW-AVG'][7]]
            df['SNOW-AVG-8'] = [x['SNOW-AVG'][8]]
            df['SNOW-AVG-9'] = [x['SNOW-AVG'][9]]
            df['SNOW-AVG-10'] = [x['SNOW-AVG'][10]]
            df['SNOW-AVG-11'] = [x['SNOW-AVG'][11]]
            df['SNOW-AVG-12'] = [x['SNOW-AVG'][12]]
        else:
            df['SNOW-AVG-1'] = np.nan
            df['SNOW-AVG-2'] = np.nan
            df['SNOW-AVG-3'] = np.nan
            df['SNOW-AVG-4'] = np.nan
            df['SNOW-AVG-5'] = np.nan
            df['SNOW-AVG-6'] = np.nan
            df['SNOW-AVG-7'] = np.nan
            df['SNOW-AVG-8'] = np.nan
            df['SNOW-AVG-9'] = np.nan
            df['SNOW-AVG-10'] = np.nan
            df['SNOW-AVG-11'] = np.nan
            df['SNOW-AVG-12'] = np.nan



        if('TAVG' in headers):
            df['TAVG-AVG-1'] = [x['TAVG-AVG'][1]]
            df['TAVG-AVG-2'] = [x['TAVG-AVG'][2]]
            df['TAVG-AVG-3'] = [x['TAVG-AVG'][3]]
            df['TAVG-AVG-4'] = [x['TAVG-AVG'][4]]
            df['TAVG-AVG-5'] = [x['TAVG-AVG'][5]]
            df['TAVG-AVG-6'] = [x['TAVG-AVG'][6]]
            df['TAVG-AVG-7'] = [x['TAVG-AVG'][7]]
            df['TAVG-AVG-8'] = [x['TAVG-AVG'][8]]
            df['TAVG-AVG-9'] = [x['TAVG-AVG'][9]]
            df['TAVG-AVG-10'] = [x['TAVG-AVG'][10]]
            df['TAVG-AVG-11'] = [x['TAVG-AVG'][11]]
            df['TAVG-AVG-12'] = [x['TAVG-AVG'][12]]
        else:
            df['TAVG-AVG-1'] = np.nan
            df['TAVG-AVG-2'] = np.nan
            df['TAVG-AVG-3'] = np.nan
            df['TAVG-AVG-4'] = np.nan
            df['TAVG-AVG-5'] = np.nan
            df['TAVG-AVG-6'] = np.nan
            df['TAVG-AVG-7'] = np.nan
            df['TAVG-AVG-8'] = np.nan
            df['TAVG-AVG-9'] = np.nan
            df['TAVG-AVG-10'] = np.nan
            df['TAVG-AVG-11'] = np.nan
            df['TAVG-AVG-12'] = np.nan


        if('TMAX' in headers):
            df['TMAX-AVG-1'] = [x['TMAX-AVG'][1]]
            df['TMAX-AVG-2'] = [x['TMAX-AVG'][2]]
            df['TMAX-AVG-3'] = [x['TMAX-AVG'][3]]
            df['TMAX-AVG-4'] = [x['TMAX-AVG'][4]]
            df['TMAX-AVG-5'] = [x['TMAX-AVG'][5]]
            df['TMAX-AVG-6'] = [x['TMAX-AVG'][6]]
            df['TMAX-AVG-7'] = [x['TMAX-AVG'][7]]
            df['TMAX-AVG-8'] = [x['TMAX-AVG'][8]]
            df['TMAX-AVG-9'] = [x['TMAX-AVG'][9]]
            df['TMAX-AVG-10'] = [x['TMAX-AVG'][10]]
            df['TMAX-AVG-11'] = [x['TMAX-AVG'][11]]
            df['TMAX-AVG-12'] = [x['TMAX-AVG'][12]]
        else:
            df['TMAX-AVG-1'] = np.nan
            df['TMAX-AVG-2'] = np.nan
            df['TMAX-AVG-3'] = np.nan
            df['TMAX-AVG-4'] = np.nan
            df['TMAX-AVG-5'] = np.nan
            df['TMAX-AVG-6'] = np.nan
            df['TMAX-AVG-7'] = np.nan
            df['TMAX-AVG-8'] = np.nan
            df['TMAX-AVG-9'] = np.nan
            df['TMAX-AVG-10'] = np.nan
            df['TMAX-AVG-11'] = np.nan
            df['TMAX-AVG-12'] = np.nan


        if('TMIN' in headers):
            df['TMIN-AVG-1'] = [x['TMIN-AVG'][1]]
            df['TMIN-AVG-2'] = [x['TMIN-AVG'][2]]
            df['TMIN-AVG-3'] = [x['TMIN-AVG'][3]]
            df['TMIN-AVG-4'] = [x['TMIN-AVG'][4]]
            df['TMIN-AVG-5'] = [x['TMIN-AVG'][5]]
            df['TMIN-AVG-6'] = [x['TMIN-AVG'][6]]
            df['TMIN-AVG-7'] = [x['TMIN-AVG'][7]]
            df['TMIN-AVG-8'] = [x['TMIN-AVG'][8]]
            df['TMIN-AVG-9'] = [x['TMIN-AVG'][9]]
            df['TMIN-AVG-10'] = [x['TMIN-AVG'][10]]
            df['TMIN-AVG-11'] = [x['TMIN-AVG'][11]]
            df['TMIN-AVG-12'] = [x['TMIN-AVG'][12]]
        else:
            df['TMIN-AVG-1'] = np.nan
            df['TMIN-AVG-2'] = np.nan
            df['TMIN-AVG-3'] = np.nan
            df['TMIN-AVG-4'] = np.nan
            df['TMIN-AVG-5'] = np.nan
            df['TMIN-AVG-6'] = np.nan
            df['TMIN-AVG-7'] = np.nan
            df['TMIN-AVG-8'] = np.nan
            df['TMIN-AVG-9'] = np.nan
            df['TMIN-AVG-10'] = np.nan
            df['TMIN-AVG-11'] = np.nan
            df['TMIN-AVG-12'] = np.nan

        df_locations = pd.concat([df_locations, df], axis=0)

    
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

### API Endpoint for the front-end

The following is a link to the full documentation of the API to access the database the DS team assembled:
https://api.citrics.io/docs

Below is a series of code examples for accessing the endpoints.

In [1]:
import json
import requests

In [2]:
# An example of the city data being pulled by City ID
# Front-end has a dictionary with City Name, ID, Lat, Lng, and Population, with that they pass this endpoint the ID. 

city_example1 = requests.get('https://api.citrics.io/jkekal6d6e5si3i2ld66d4dl/citydata/17417')
city_example1 = city_example1.json()
city_example1

{'_id': 17417,
 'name_with_com': 'Fort Worth, TX',
 'city_no_st': 'Fort Worth',
 'census_full_name': 'Fort Worth city, Texas',
 'City': 'Fort Worth TX',
 'State': 'Texas',
 'st': 'TX',
 'geo_id': '1600000US4827000',
 'Latitude': 32.7812,
 'Longitude': -97.3472,
 'Total Population': 855786.0,
 'Population Male & Female': {'Male': 49.0, 'Female': 51.0},
 'Age Distribution': {'Under 5': 8.1,
  '5 to 9': 8.2,
  '10 to 14': 7.6,
  '15 to 19': 7.2,
  '20 to 24': 7.2,
  '25 to 34': 16.3,
  '35 to 44': 14.0,
  '45 to 54': 12.4,
  '55 to 59': 5.4,
  '60 to 64': 4.4,
  '65 to 74': 5.6,
  '75 to 84': 2.6,
  '85 years and over': 1.1},
 'Median Age': 32.2,
 'Population under 18': 28.3,
 'Population over 16': 74.7,
 'Ethnicity': {'White': 40.2,
  'Hispanic or Latino': 34.8,
  'African American': 18.4,
  'American Indian': 0.2,
  'Asian': 3.8,
  'Pacific Islander': 0.1,
  'other race': 0.2,
  'Two or more races': 2.3},
 'Marital Status': {'Males': {'Never Married': 37.2,
   'Currently Married': 48.1,

In [3]:
# An example of the city suggestions being shown for names 

city_example2 = requests.get('https://api.citrics.io/jkekal6d6e5si3i2ld66d4dl/matchcity/fort-worth-tx')
city_example2 = city_example2.json()
city_example2

{'Fort Worth, TX': {'ID': 17417,
  'lat': 32.7812,
  'lng': -97.3472,
  'state': 'Texas',
  'st': 'TX',
  'population': 855786},
 'Wortham, TX': {'ID': 17052,
  'lat': 31.7897,
  'lng': -96.461,
  'state': 'Texas',
  'st': 'TX',
  'population': 1176}}

In [4]:
city_example3 = requests.get('https://api.citrics.io/jkekal6d6e5si3i2ld66d4dl/ip_to_city/136.34.38.43') #Note: IPv4
city_example3 = city_example3.json()
city_example3

{'_id': 9619,
 'name_with_com': 'Kansas City, MO',
 'city_no_st': 'Kansas City',
 'census_full_name': 'Kansas City city, Missouri',
 'City': 'Kansas City MO',
 'State': 'Missouri',
 'st': 'MO',
 'geo_id': '1600000US2938000',
 'Latitude': 39.1239,
 'Longitude': -94.5541,
 'Total Population': 481417.0,
 'Population Male & Female': {'Male': 48.7, 'Female': 51.3},
 'Age Distribution': {'Under 5': 7.0,
  '5 to 9': 6.4,
  '10 to 14': 6.4,
  '15 to 19': 5.6,
  '20 to 24': 7.2,
  '25 to 34': 17.2,
  '35 to 44': 13.2,
  '45 to 54': 12.8,
  '55 to 59': 6.4,
  '60 to 64': 5.6,
  '65 to 74': 7.3,
  '75 to 84': 3.5,
  '85 years and over': 1.7},
 'Median Age': 35.2,
 'Population under 18': 23.1,
 'Population over 16': 79.1,
 'Ethnicity': {'White': 55.5,
  'Hispanic or Latino': 10.2,
  'African American': 28.5,
  'American Indian': 0.3,
  'Asian': 2.7,
  'Pacific Islander': 0.1,
  'other race': 0.2,
  'Two or more races': 2.6},
 'Marital Status': {'Males': {'Never Married': 42.2,
   'Currently Marrie